In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import re 

In [38]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [175]:
df = pd.read_csv('SIAPE_ativos_federal_raca_orgao_juridico.csv')
df

,ano,mes_base,nat_jur,raca,vinculos,total
0,1999,Fev,Autarquia Federal,Branca,100527,188092
1,1999,Fev,Autarquia Federal,Não informada,32263,188092
2,1999,Fev,Autarquia Federal,Negra,48488,188092
3,1999,Fev,Autarquia Federal,Outros,6814,188092
4,1999,Fev,Emp Pub S A Cap Fechado,Branca,9,18
...,...,...,...,...,...,...
340,2020,Fev,Fundacao Federal,Outros,1993,73138
341,2020,Fev,Poder Executivo Federal,Branca,91144,173994
342,2020,Fev,Poder Executivo Federal,Não informada,11517,173994
343,2020,Fev,Poder Executivo Federal,Negra,65508,173994


In [176]:
df.drop(df[df['nat_jur'] == 'Emp Pub  S A  Cap Fechado'].index, inplace=True)


In [177]:
df['nat_jur'].unique()

array(['Autarquia Federal', 'Fundacao Federal', 'Poder Executivo Federal'],
      dtype=object)

In [178]:
df['mes_base'].unique()

array(['Fev'], dtype=object)

In [179]:
df['raca'].unique()

array(['Branca', 'Não informada', 'Negra', 'Outros'], dtype=object)

In [180]:
def transformar(nome):
    nome=re.sub(r"\bFev\b", "Fevereivo", nome)
    nome = re.sub(r"\bFundacao\b", "Fundação", nome)
    return nome

In [181]:
df['mes_base'] =df['mes_base'].apply(transformar)

In [182]:
df

,ano,mes_base,nat_jur,raca,vinculos,total
0,1999,Fevereivo,Autarquia Federal,Branca,100527,188092
1,1999,Fevereivo,Autarquia Federal,Não informada,32263,188092
2,1999,Fevereivo,Autarquia Federal,Negra,48488,188092
3,1999,Fevereivo,Autarquia Federal,Outros,6814,188092
8,1999,Fevereivo,Fundacao Federal,Branca,36332,89582
...,...,...,...,...,...,...
340,2020,Fevereivo,Fundacao Federal,Outros,1993,73138
341,2020,Fevereivo,Poder Executivo Federal,Branca,91144,173994
342,2020,Fevereivo,Poder Executivo Federal,Não informada,11517,173994
343,2020,Fevereivo,Poder Executivo Federal,Negra,65508,173994


In [183]:
df = df.rename(columns={'nat_jur': 'natureza_juridica', 'raca' : 'cor_raca', 'vinculos':'quantidade_vinculos'}) 


In [184]:
df

,ano,mes_base,natureza_juridica,cor_raca,quantidade_vinculos,total
0,1999,Fevereivo,Autarquia Federal,Branca,100527,188092
1,1999,Fevereivo,Autarquia Federal,Não informada,32263,188092
2,1999,Fevereivo,Autarquia Federal,Negra,48488,188092
3,1999,Fevereivo,Autarquia Federal,Outros,6814,188092
8,1999,Fevereivo,Fundacao Federal,Branca,36332,89582
...,...,...,...,...,...,...
340,2020,Fevereivo,Fundacao Federal,Outros,1993,73138
341,2020,Fevereivo,Poder Executivo Federal,Branca,91144,173994
342,2020,Fevereivo,Poder Executivo Federal,Não informada,11517,173994
343,2020,Fevereivo,Poder Executivo Federal,Negra,65508,173994


In [188]:
df = df.drop(['total'], axis = 1)

In [189]:
df

,ano,mes_base,natureza_juridica,cor_raca,quantidade_vinculos
0,1999,Fevereivo,Autarquia Federal,Branca,100527
1,1999,Fevereivo,Autarquia Federal,Não informada,32263
2,1999,Fevereivo,Autarquia Federal,Negra,48488
3,1999,Fevereivo,Autarquia Federal,Outros,6814
8,1999,Fevereivo,Fundacao Federal,Branca,36332
...,...,...,...,...,...
340,2020,Fevereivo,Fundacao Federal,Outros,1993
341,2020,Fevereivo,Poder Executivo Federal,Branca,91144
342,2020,Fevereivo,Poder Executivo Federal,Não informada,11517
343,2020,Fevereivo,Poder Executivo Federal,Negra,65508


Subindo para o GBQ

In [190]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [191]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes_base','STRING',description='Mês de quando a observação foi executada'),
        bigquery.SchemaField('natureza_juridica','STRING',description='Natureza jurídica referente ao cargo da observação'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Número total de vinculos observados')]

In [192]:
table_ref = dataset_ref.table('SIAPE_ativos_federal_raca_orgao_juridico')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=23fc6cf3-3ee6-4bdf-8405-5d647221fe2c>